In [7]:
import glob
import pandas as pd
# Use the glob function to match and retrieve the file paths
file_paths = glob.glob('/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/**/*.parquet')

# Now, file_paths contains a list of file paths matching the pattern
print(file_paths)

['/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2018/cm_actuals_2018.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2020/cm_actuals_2020.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2021/cm_actuals_2021.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2019/cm_actuals_2019.parquet']


In [8]:
for i in range(0,4):
    df = pd.read_parquet(file_paths[i])
    print(df)

                     outcome
month_id country_id         
457      1               0.0
         2               0.0
         3               0.0
         4               0.0
         5               0.0
...                      ...
468      242             0.0
         243             0.0
         244             0.0
         245             9.0
         246            39.0

[2292 rows x 1 columns]
                     outcome
month_id country_id         
481      1               0.0
         2               0.0
         3               0.0
         4               0.0
         5               0.0
...                      ...
492      242             0.0
         243             0.0
         244             0.0
         245             0.0
         246             0.0

[2292 rows x 1 columns]
                     outcome
month_id country_id         
493      1               0.0
         2               0.0
         3               0.0
         4               0.0
         5            

In [14]:
from sklearn.model_selection import ShuffleSplit


In [15]:
shuffle_split = ShuffleSplit(n_splits=1000, test_size=len(df), random_state=42)


In [16]:
shuffle_split

ShuffleSplit(n_splits=1000, random_state=42, test_size=2292, train_size=None)

In [17]:
for i, (train_idx, _) in enumerate(shuffle_split.split(df)):
    bootstrapped_sample = df.iloc[train_idx]
    # Rename the columns to indicate the bootstrap iteration
    bootstrapped_sample.columns = [f'Bootstrap_{i+1} - {col}' for col in bootstrapped_sample.columns]
    bootstrapped_df = pd.concat([bootstrapped_df, bootstrapped_sample], axis=1)

ValueError: test_size=2292 should be either positive and smaller than the number of samples 2292 or a float in the (0, 1) range

In [31]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import glob
import pyarrow.parquet as pq  

# Use the glob function to match and retrieve the file paths
file_paths = glob.glob('/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/**/*.parquet')

# Now, file_paths contains a list of file paths matching the pattern
print(file_paths)
year = 2018
for x in file_paths:
    df = pd.read_parquet(x)
    # Create the original DataFrame
    original_df = pd.DataFrame(df)

    # Initialize an empty DataFrame to store the bootstrapped draws
    bootstrapped_df = pd.DataFrame()

    # Define the number of bootstraps
    n_bootstraps = 1000

    # Perform bootstrapping for each unique combination of country_id and month_id
    for (country_id, month_id), group in original_df.groupby(['country_id', 'month_id']):
        bootstrapped_samples = []
        for _ in range(n_bootstraps):
            bootstrapped_sample = resample(group['outcome'], replace=True)
            bootstrapped_samples.append(bootstrapped_sample.mean())
        bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'country_id': [country_id] * n_bootstraps,
                                                                'month_id': [month_id] * n_bootstraps,
                                                                'draws': bootstrapped_samples})])

    # Set country_id and month_id as indexes
    bootstrapped_df.set_index(['country_id', 'month_id'], inplace=True)
    bootstrapped_df.to_parquet(f'/Users/noorainkazmi/Documents/bootstrap/cm_bootstrapped_{year}',engine='pyarrow')
    # Display the resulting DataFrame
    print(bootstrapped_df)
    year = year + 1


['/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2018/cm_actuals_2018.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2020/cm_actuals_2020.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2021/cm_actuals_2021.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2019/cm_actuals_2019.parquet']
                     draws
country_id month_id       
1          457         0.0
           457         0.0
           457         0.0
           457         0.0
           457         0.0
...                    ...
246        468        39.0
           468        39.0
           468        39.0
           468        39.0
           468        39.0

[2292000 rows x 1 columns]
                     draws
country_id month_id       
1          481         0.0
           481         0.0
           481         0.0
           481         0.0
           481         0.0

In [29]:
bootstrapped_df

draws
country_id month_id       
1          457         0.0
           457         0.0
           457         0.0
           457         0.0
           457         0.0
...                    ...
246        468        39.0
           468        39.0
           468        39.0
           468        39.0
           468        39.0

[2292000 rows x 1 columns]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import glob
import pyarrow.parquet as pq  
def bootstrap_cm_pgm(foldername):
    # Use the glob function to match and retrieve the file paths
    file_paths = glob.glob(foldername+'actuals/cm/**/*.parquet')

    # Now, file_paths contains a list of file paths matching the pattern
    #print(file_paths)
    year = 2018
    for x in file_paths:
        df = pd.read_parquet(x)
        # Create the original DataFrame
        original_df = pd.DataFrame(df)

        # Initialize an empty DataFrame to store the bootstrapped draws
        bootstrapped_df = pd.DataFrame()

        # Define the number of bootstraps
        n_bootstraps = 1000

        # Perform bootstrapping for each unique combination of country_id and month_id
        for (country_id, month_id), group in original_df.groupby(['country_id', 'month_id']):
            bootstrapped_samples = []
            for _ in range(n_bootstraps):
                bootstrapped_sample = resample(group['outcome'], replace=True)
                bootstrapped_samples.append(bootstrapped_sample.mean())
            bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'country_id': [country_id] * n_bootstraps,
                                                                    'month_id': [month_id] * n_bootstraps,
                                                                    'draws': bootstrapped_samples})])

        # Set country_id and month_id as indexes
        bootstrapped_df.set_index(['country_id', 'month_id'], inplace=True)
        bootstrapped_df.to_parquet(f'{foldername}/bootstrap/cm_bootstrapped_{year}',engine='pyarrow')
        # Display the resulting DataFrame
        year = year+1
        
    file_paths = glob.glob(foldername+'actuals/pgm/**/*.parquet')

    # Now, file_paths contains a list of file paths matching the pattern
    #print(file_paths)
    year = 2018
    for x in file_paths:
        df = pd.read_parquet(x)
        # Create the original DataFrame
        original_df = pd.DataFrame(df)

        # Initialize an empty DataFrame to store the bootstrapped draws
        bootstrapped_df = pd.DataFrame()

        # Define the number of bootstraps
        n_bootstraps = 1000

        # Perform bootstrapping for each unique combination of country_id and month_id
        for (country_id, month_id), group in original_df.groupby(['priogrid_gid', 'month_id']):
            bootstrapped_samples = []
            for _ in range(n_bootstraps):
                bootstrapped_sample = resample(group['outcome'], replace=True)
                bootstrapped_samples.append(bootstrapped_sample.mean())
            bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'priogrid_gid': [priogrid_gid] * n_bootstraps,
                                                                    'month_id': [month_id] * n_bootstraps,
                                                                    'draws': bootstrapped_samples})])

        # Set country_id and month_id as indexes
        bootstrapped_df.set_index(['priogrid_gid', 'month_id'], inplace=True)
        bootstrapped_df.to_parquet(f'{foldername}/bootstrap/pgm_bootstrapped_{year}',engine='pyarrow')
        year = year + 1
        # Display the resulting DataFrame

In [34]:
df = pd.read_parquet('/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/submissions/benchmark_bootstrap_expanded/cm/test_window_2018/bm_cm_bootstrap_expanded_2018.parquet')

In [35]:
df

outcome
month_id country_id draw         
457      1          0           0
                    1           0
                    2           0
                    3           0
                    4           0
...                           ...
468      246        995         0
                    996         0
                    997         0
                    998         0
                    999         0

[2292000 rows x 1 columns]

In [37]:
import pandas as pd
import numpy as np

# Sample data for demonstration
data = {
    'month_id': [1, 2, 1, 2, 1, 2],
    'country_id': ['A', 'A', 'B', 'B', 'C', 'C'],
    'outcome': [10, 15, 20, 25, 30, 35]
}

# Create the original DataFrame
original_df = pd.read_parquet(file_paths[0])

# Initialize an empty DataFrame to store the bootstrapped draws
bootstrapped_df = pd.DataFrame()

# Define the number of bootstraps
n_bootstraps = 1000

# Perform bootstrapping for each unique combination of month_id and country_id
for (month_id, country_id), group in original_df.groupby(['month_id', 'country_id']):
    for draw in range(n_bootstraps):
        bootstrapped_sample = group['outcome'].sample(frac=1, replace=True)
        bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'month_id': [month_id],
                                                                   'country_id': [country_id],
                                                                   'draw': [draw],
                                                                   'outcome': [bootstrapped_sample.mean()]})])

# Set month_id, country_id, and draw as indexes
bootstrapped_df.set_index(['month_id', 'country_id', 'draw'], inplace=True)

# Display the resulting DataFrame
print(bootstrapped_df)
